# OpenAI Responses API - Data and Connectors Demo

This notebook demonstrates the powerful **File Search** capability of the OpenAI Responses API, which allows models to retrieve information from your uploaded documents through semantic and keyword search.

The File Search tool offers several key advantages:
- **Semantic search** - Find relevant information by meaning, not just keywords
- **Vector store integration** - Scalable knowledge base management
- **Automatic citations** - Get references to source documents in responses
- **Hosted solution** - No need to implement search infrastructure yourself
- **Retrieval customization** - Control search results, filtering, and metadata
- **Multiple file formats** - Support for PDFs, docs, code files, and more

This notebook walks through setting up vector stores, uploading documents, and using file search with the Responses API to create AI applications that can reason over your data.

---


### Setup and Authentication

First, we need to set up our OpenAI API credentials. We'll use `getpass` to securely input the API key without exposing it in the notebook.


In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Now we'll initialize the OpenAI client and import the additional libraries we'll need for file handling and requests.


In [2]:
from openai import OpenAI
import requests
from io import BytesIO

client = OpenAI()

### Setting Up File Search: Vector Stores and File Upload

Before we can use file search with the Responses API, we need to create a knowledge base. This involves three key steps:

1. **Upload files** to the OpenAI File API
2. **Create a vector store** to organize our knowledge base
3. **Add files to the vector store** for semantic search

Let's start by creating a helper function that can handle both local files and URLs, similar to the example in the OpenAI documentation.


In [3]:
def create_file(client, file_path):
    """
    Upload a file to OpenAI's File API.
    Supports both local file paths and URLs.
    
    Args:
        client: OpenAI client instance
        file_path: Path to local file or URL to download from
        
    Returns:
        str: File ID from OpenAI
    """
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        print(f"Downloading file from URL: {file_path}")
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        print(f"Uploading local file: {file_path}")
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    
    print(f"File uploaded successfully. File ID: {result.id}")
    return result.id

Now let's upload your local PDF file about embeddings from the data folder. This will serve as our knowledge base for testing file search functionality.


In [4]:
# Upload your local PDF file from the data folder
file_id = create_file(client, "/home/chris/Code/AI Makerspace/Events/OpenAI Responses API/data/Embedding-Based.pdf")

Uploading local file: /home/chris/Code/AI Makerspace/Events/OpenAI Responses API/data/Embedding-Based.pdf


FileNotFoundError: [Errno 2] No such file or directory: '/home/chris/Code/AI Makerspace/Events/OpenAI Responses API/data/Embedding-Based.pdf'

Next, we need to create a **vector store**. Think of this as a container that organizes your uploaded files for semantic search. The vector store will automatically process your documents and create embeddings for efficient retrieval.


In [ ]:
# Create a vector store for our knowledge base
vector_store = client.vector_stores.create(
    name="AI_Makerspace_Knowledge_Base"
)

print(f"Vector store created successfully!")
print(f"Vector Store ID: {vector_store.id}")
print(f"Name: {vector_store.name}")
print(f"Status: {vector_store.status}")

Vector store created successfully!
Vector Store ID: vs_68c1aea8ca8081918962708d7420ffde
Name: AI_Makerspace_Knowledge_Base
Status: completed


Now we'll add our uploaded file to the vector store. This step tells OpenAI to process the document and make it available for semantic search.


In [ ]:
# Add the uploaded file to our vector store
result = client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

print(f"File added to vector store successfully!")
print(f"File ID: {result.id}")
print(f"Status: {result.status}")
print(f"Created at: {result.created_at}")

File added to vector store successfully!
File ID: file-WrMJNKgKCN71PzTs2BQrE4
Status: in_progress
Created at: 1757523628


We need to wait for the file processing to complete before we can use it for search. Let's check the status and wait until it's ready. The file needs to be in `completed` status for search to work properly.


In [ ]:
import time

def wait_for_file_processing(client, vector_store_id, max_wait_time=300):
    """
    Wait for all files in a vector store to be processed.
    
    Args:
        client: OpenAI client instance
        vector_store_id: ID of the vector store to check
        max_wait_time: Maximum time to wait in seconds (default: 5 minutes)
    
    Returns:
        bool: True if all files are completed, False if timeout
    """
    start_time = time.time()
    
    while time.time() - start_time < max_wait_time:
        # Check the status of files in the vector store
        files = client.vector_stores.files.list(vector_store_id=vector_store_id)
        
        statuses = [file.status for file in files.data]
        print(f"File statuses: {statuses}")
        
        if all(status == "completed" for status in statuses):
            print("✅ All files processed successfully!")
            return True
        elif any(status == "failed" for status in statuses):
            print("❌ One or more files failed to process")
            return False
        else:
            print("⏳ Still processing... waiting 10 seconds")
            time.sleep(10)
    
    print("⏰ Timeout waiting for file processing")
    return False

# Wait for our file to be processed
wait_for_file_processing(client, vector_store.id)

File statuses: ['completed']
✅ All files processed successfully!


True

### Basic File Search with the Responses API

Now that our knowledge base is set up, we can use the **file search** tool with the Responses API! This is where the magic happens - the model can automatically search through your documents and provide answers with citations.

Key features of file search:
- **Automatic tool calling** - The model decides when to search your files
- **Semantic understanding** - Finds relevant information by meaning, not just keywords  
- **Source citations** - Get references to specific documents in the response
- **Multiple output types** - Both search calls and final messages with citations


In [ ]:
# Use file search with the Responses API
response = client.responses.create(
    model="gpt-5",
    input="What is the main problem with dense vector embeddings?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

print("🔍 Response from file search:")
print("=" * 50)
print(response.output_text)

🔍 Response from file search:
Dense (single‑vector) embeddings have a fundamental capacity limit: because their representation power is bounded by the embedding dimension, they cannot realize all possible top‑k relevance combinations; for any fixed dimension d there exist queries/relevance patterns they will fail to return, no matter how they’re trained . This limitation can be formalized via sign‑rank: some relevance matrices require higher dimensionality than the model has, so dense embeddings cannot capture them exactly . As tasks demand more combinations (e.g., instruction‑based retrieval), the dimensionality needed grows rapidly and becomes impractical at scale .


Let's examine the full response structure to understand what file search returns. The response contains multiple output items including the search call details and the final message with citations.


In [ ]:
# Examine the full response structure
print("📊 Full Response Analysis:")
print("=" * 50)

for i, output_item in enumerate(response.output):
    print(f"\n📋 Output Item {i + 1}:")
    print(f"   Type: {output_item.type}")
    print(f"   ID: {output_item.id}")
    
    if output_item.type == "file_search_call":
        print(f"   Status: {output_item.status}")
        print(f"   Queries: {output_item.queries}")
        
    elif output_item.type == "message":
        print(f"   Role: {output_item.role}")
        print(f"   Content items: {len(output_item.content)}")
        
        # Check for citations in the message content
        for content_item in output_item.content:
            if hasattr(content_item, 'annotations') and content_item.annotations:
                print(f"   📚 Found {len(content_item.annotations)} citations")
                for j, annotation in enumerate(content_item.annotations):
                    if annotation.type == "file_citation":
                        print(f"      Citation {j + 1}: {annotation.filename}")


📊 Full Response Analysis:

📋 Output Item 1:
   Type: reasoning
   ID: rs_68c1b00540f081a081e58d4e59bff7ab047aaa5f6e571614

📋 Output Item 2:
   Type: file_search_call
   ID: fs_68c1b008dea081a0a679634f7078f796047aaa5f6e571614
   Status: completed
   Queries: ['What is the main problem with dense vector embeddings?', 'limitations of dense vector embeddings main problem', 'dense embeddings problem opacity interpretability sparsity', 'dense vs sparse embeddings advantages disadvantages', 'What is the problem with dense vector representations in NLP?']

📋 Output Item 3:
   Type: reasoning
   ID: rs_68c1b00b202881a0ace83f23d5e7eb18047aaa5f6e571614

📋 Output Item 4:
   Type: message
   ID: msg_68c1b01ec16881a0b1554faa33adee5c047aaa5f6e571614
   Role: assistant
   Content items: 1
   📚 Found 3 citations
      Citation 1: Embedding-Based.pdf
      Citation 2: Embedding-Based.pdf
      Citation 3: Embedding-Based.pdf


### Including Search Results in the Response

By default, the file search call doesn't return the actual search results - only the final answer with citations. However, you can include the search results using the `include` parameter. This is useful for understanding what information was retrieved and how the model used it.


In [ ]:
# Include search results in the response
response_with_results = client.responses.create(
    model="gpt-5",
    input="What are the key limitations of dense vector embeddings?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }],
    include=["file_search_call.results"]
)

print("🔍 Response with search results:")
print("=" * 50)
print(response_with_results.output_text)

# Examine the search results
for output_item in response_with_results.output:
    if output_item.type == "file_search_call" and hasattr(output_item, 'search_results'):
        if output_item.search_results:
            print(f"\n📄 Search Results Found:")
            print(f"Number of results: {len(output_item.search_results)}")
            
            for i, result in enumerate(output_item.search_results):
                print(f"\nResult {i + 1}:")
                print(f"  Score: {result.score}")
                print(f"  Content preview: {result.content[:200]}...")
        else:
            print("\n📄 No search results returned (search_results is None)")


🔍 Response with search results:
Here are the main limitations practitioners run into with dense, single‑vector embeddings:

- Finite representational capacity tied to dimension d: For any fixed d, there exist relevance patterns (top‑k sets) that a single-vector embedder simply cannot realize—no amount of training data fixes this. This follows from sign‑rank bounds on the relevance (qrel) matrix and yields an intrinsic ceiling on what a d‑dimensional model can retrieve exactly .

- Breakdowns as combinations grow: As tasks demand returning many different combinations of relevant documents (e.g., instruction‑following or reasoning queries that connect previously unrelated items), dense retrievers hit these capacity limits. Empirically, even state‑of‑the‑art models struggle on LIMIT, a simple dataset constructed to stress such combinations, with recall remaining low despite trivial content  . The “dense” qrel pattern that maximizes combinations is especially damaging across models .

- Yo

### Retrieval Customization

The file search tool offers several customization options to optimize performance and results:

1. **Limiting results** - Control the number of search results to reduce tokens and latency
2. **Metadata filtering** - Filter search results based on file attributes
3. **Search quality vs. performance** - Balance between comprehensive results and response speed

Let's explore these customization options:


#### Limiting the Number of Results

By default, file search may retrieve many results. You can limit this using `max_num_results` to reduce token usage and improve response time, though this may impact answer quality.


In [ ]:
# Limit the number of search results
response_limited = client.responses.create(
    model="gpt-5",
    input="What is the main problem with dense vector embeddings?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 2  # Limit to 2 results for faster, more focused responses
    }],
    include=["file_search_call.results"]
)

print("🎯 Limited Results Response:")
print("=" * 50)
print(response_limited.output_text)

# Show how many results were actually used
for output_item in response_limited.output:
    if output_item.type == "file_search_call":
        if hasattr(output_item, 'search_results') and output_item.search_results:
            print(f"\n📊 Used {len(output_item.search_results)} search results (limited to 2)")

🎯 Limited Results Response:
The core problem is limited expressivity: a single fixed‑dimensional dense vector can’t encode all the combinations of document–query relevance you may need. For any chosen dimension, there exist top‑k relevance patterns that cannot be realized—no matter how you train the model—so as corpora and tasks grow, dense embeddings hit a hard capacity limit unless you dramatically increase dimensionality . In fact, theoretical bounds show the required dimensionality can be much larger than what’s practical for real IR problems, which leads to recall errors and false positives at scale .


#### Practical Example: Adding Local Files to Your Knowledge Base

Let's demonstrate how to add local files from your project to create a more comprehensive knowledge base. We'll check if there are any PDF files in the data directory and add them to our vector store.


In [ ]:
import os
import glob

# Check for local PDF files in the data directory
data_dir = "./data"
pdf_files = []

if os.path.exists(data_dir):
    pdf_files = glob.glob(os.path.join(data_dir, "*.pdf"))
    print(f"📁 Found {len(pdf_files)} PDF files in {data_dir}:")
    for pdf_file in pdf_files:
        print(f"   - {os.path.basename(pdf_file)}")
else:
    print(f"📁 Data directory {data_dir} not found")

# Upload and add local files to the vector store
local_file_ids = []
for pdf_file in pdf_files[:1]:  # Let's add just the first PDF to avoid too much processing time
    try:
        print(f"\n📤 Uploading {os.path.basename(pdf_file)}...")
        local_file_id = create_file(client, pdf_file)
        local_file_ids.append(local_file_id)
        
        # Add to vector store
        result = client.vector_stores.files.create(
            vector_store_id=vector_store.id,
            file_id=local_file_id
        )
        print(f"✅ Added to vector store successfully!")
        
    except Exception as e:
        print(f"❌ Error processing {pdf_file}: {e}")

if local_file_ids:
    print(f"\n⏳ Waiting for new files to be processed...")
    wait_for_file_processing(client, vector_store.id)
else:
    print("\n💡 No local files were added. You can place PDF files in the ./data directory to test with your own documents!")

📁 Found 2 PDF files in ./data:
   - Embedding-Based.pdf
   - Harnessing Embeddings.pdf

📤 Uploading Embedding-Based.pdf...
Uploading local file: ./data/Embedding-Based.pdf
File uploaded successfully. File ID: file-EunX48ia7QfQNNhXppgJgK
✅ Added to vector store successfully!

⏳ Waiting for new files to be processed...
File statuses: ['in_progress', 'completed']
⏳ Still processing... waiting 10 seconds
File statuses: ['completed', 'completed']
✅ All files processed successfully!


### Multi-Document Search and Comparison

Now that we have multiple documents in our knowledge base, we can ask questions that require searching across different sources. This demonstrates the power of semantic search for complex queries.


In [ ]:
# Ask a complex question that might require multiple documents
complex_response = client.responses.create(
    model="gpt-5",
    input="Compare and contrast different approaches to AI research and development mentioned in the documents. What are the key methodologies and their benefits?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }],
    include=["file_search_call.results"]
)

print("🔬 Complex Multi-Document Query:")
print("=" * 60)
print(complex_response.output_text)

# Show which documents were referenced
print("\n📚 Documents Referenced:")
print("=" * 30)
for output_item in complex_response.output:
    if output_item.type == "message":
        for content_item in output_item.content:
            if hasattr(content_item, 'annotations') and content_item.annotations:
                unique_files = set()
                for annotation in content_item.annotations:
                    if annotation.type == "file_citation":
                        unique_files.add(annotation.filename)
                
                for filename in unique_files:
                    print(f"📄 {filename}")

🔬 Complex Multi-Document Query:
Below is a concise map of the main AI R&D approaches and methodologies discussed in the documents, with their benefits and trade‑offs.

1) Theoretical analysis of model limits
- What it is: Use tools from communication complexity and sign‑rank to prove lower bounds on what single‑vector (dense) embedding models can represent in top‑k retrieval; relates but contrasts with geometric intuitions like order‑k Voronoi regions. Also clarifies why triangle‑inequality arguments don’t apply when using cosine similarity (non‑metric)  .
- Benefits:
  - Gives rigorous, architecture‑agnostic limits for single‑vector embeddings across modalities: for a fixed embedding dimension d, some top‑k combinations simply cannot be returned by any query  .
  - Guides system design by explaining why scaling tasks that require many combinatorial combinations will eventually hit representation ceilings .
- Trade‑offs:
  - Results target the dominant single‑vector paradigm; extension

### Supported File Formats

The file search tool supports a wide variety of file formats. Here's a summary of what you can upload:

**Text and Code Files:**
- `.py`, `.js`, `.ts`, `.java`, `.cpp`, `.c`, `.cs`, `.go`, `.php`, `.rb`, `.sh`
- `.html`, `.css`, `.json`, `.md`, `.txt`, `.tex`

**Document Formats:**
- `.pdf` - PDF documents
- `.doc`, `.docx` - Microsoft Word documents  
- `.pptx` - PowerPoint presentations

**Requirements:**
- For text files, encoding must be UTF-8, UTF-16, or ASCII
- Files are processed automatically for semantic search
- Maximum file size and token limits apply (check OpenAI documentation for current limits)


## Summary

The OpenAI Responses API with File Search represents a powerful leap forward in building AI applications that can reason over your data. Key takeaways:

✅ **Hosted solution** - No need to manage your own vector database or search infrastructure  
✅ **Semantic search** - Find information by meaning, not just keywords  
✅ **Automatic citations** - Get references to source documents in responses  
✅ **Multiple file formats** - Support for PDFs, docs, code files, and more  
✅ **Retrieval customization** - Control search results and performance  
✅ **Easy integration** - Simple API that works seamlessly with the Responses API  


# MCP (Model Context Protocol) and Connectors

In addition to file search, the OpenAI Responses API supports **MCP (Model Context Protocol)** servers and **Connectors** that give models the ability to connect to and control external services. This section demonstrates:

## Two Types of External Integrations:

### 1. **Connectors** 
- OpenAI-maintained MCP wrappers for popular services
- Pre-built integrations like Google Workspace, Dropbox, Microsoft 365
- OAuth-based authentication
- Hosted by OpenAI with guaranteed reliability

### 2. **Remote MCP Servers**
- Third-party servers implementing the MCP protocol
- Can be any server on the public Internet
- Custom tools and capabilities
- Examples: GitHub, Stripe, custom business tools

## Key Benefits:
- **Automatic tool discovery** - Models learn available tools dynamically
- **Approval workflows** - Control what data is shared with external services  
- **Real-time capabilities** - Access live data and perform actions
- **Extensible** - Connect to virtually any service with an MCP server

---

⚠️  **Please obtain a Google OAuth token first!**

📝 **Steps to get a token:**
1. Go to [Google OAuth Playground](https://developers.google.com/oauthplayground/)
2. Enter scope: `https://www.googleapis.com/auth/calendar.events`
3. Authorize APIs and exchange for token
4. Replace the token in the code above

In [5]:
os.environ["GOOGLE_OAUTH_TOKEN"] = getpass.getpass("Enter your Google OAuth token: ")

In [6]:
# Google Calendar Connector Example
# Note: You'll need to obtain an OAuth access token from Google
# For testing, use Google's OAuth 2.0 Playground: https://developers.google.com/oauthplayground/

# Example OAuth scope for Google Calendar:
# https://www.googleapis.com/auth/calendar.events

def demonstrate_google_calendar_connector(token):
    """
    Demonstrate Google Calendar Connector functionality.
    This example shows how to query calendar events using the Responses API.
    """
    
    try:
        # Query today's calendar events
        response = client.responses.create(
            model="gpt-5",
            tools=[{
                "type": "mcp",
                "server_label": "google_calendar",
                "connector_id": "connector_googlecalendar",
                "authorization": token,
                "require_approval": "never"  # Set to "always" for production
            }],
            input="What events do I have on my Google Calendar for Friday, September 12th? Please provide details about each event including time, title, and any important information."
        )
        
        print("📅 Google Calendar Response:")
        print("=" * 50)
        print(response.output_text)
        
        return response
        
    except Exception as e:
        print(f"❌ Error with Google Calendar Connector: {e}")
        return None

# Demonstrate the connector (will show setup instructions if no token provided)
calendar_response = demonstrate_google_calendar_connector(os.environ["GOOGLE_OAUTH_TOKEN"])

📅 Google Calendar Response:
I can pull that from your Google Calendar, but I need to confirm the year and time zone.

- Friday, September 12 most recently occurred in 2025. Is that the date you mean, or a different year?
- What time zone should I use for the day’s boundaries (e.g., your calendar’s default)?

Note: I tried to fetch your events just now but ran into a temporary connection error. If you confirm the year and time zone, I’ll try again immediately and return each event’s time, title, attendees, location/meet link, and notes.


#### Advanced Google Calendar Operations

The Google Calendar Connector supports several sophisticated operations that can be combined for powerful calendar-based workflows:


In [7]:
def demonstrate_advanced_calendar_operations(token):
    """
    Demonstrate advanced Google Calendar operations including:
    - Weekly schedule analysis
    - Meeting conflict detection
    - Event filtering and search
    """
    try:
        # Advanced calendar analysis query
        response = client.responses.create(
            model="gpt-5",
            tools=[{
                "type": "mcp",
                "server_label": "google_calendar",
                "connector_id": "connector_googlecalendar", 
                "authorization": token,
                "require_approval": "never",
                # Limit to specific tools for focused functionality
                "allowed_tools": ["search_events", "read_event"]
            }],
            input="""Analyze my Google Calendar for this week and provide:
            1. A summary of my busiest days
            2. Any potential scheduling conflicts
            3. Recommendations for optimal meeting times
            4. Total hours of scheduled meetings
            
            Please be thorough in your analysis and provide actionable insights."""
        )
        
        print("📊 Advanced Calendar Analysis:")
        print("=" * 50)
        print(response.output_text)
        
        # Examine the MCP calls that were made
        print("\n🔍 MCP Operations Performed:")
        print("=" * 30)
        
        for i, output_item in enumerate(response.output):
            if output_item.type == "mcp_call":
                print(f"\n📋 MCP Call {i + 1}:")
                print(f"   Tool: {output_item.name}")
                print(f"   Status: {'✅ Success' if not output_item.error else '❌ Error'}")
                if output_item.error:
                    print(f"   Error: {output_item.error}")
                else:
                    # Show a preview of the output
                    output_preview = str(output_item.output)[:200]
                    print(f"   Output preview: {output_preview}...")
        
        return response
        
    except Exception as e:
        print(f"❌ Error with advanced calendar operations: {e}")
        return None

# Run advanced calendar analysis
advanced_calendar_response = demonstrate_advanced_calendar_operations(os.environ["GOOGLE_OAUTH_TOKEN"])


📊 Advanced Calendar Analysis:
I’m ready to do this, but I couldn’t pull your events due to a temporary Google Calendar connection error. I attempted to fetch your events for this week (Oct 27–Nov 2, 2025) and received a 500 error from the calendar service. If you’d like, I can retry immediately.

Before I retry, please confirm a few details so I can be precise:
- Timezone: Is America/Los_Angeles correct? If not, what timezone should I use?
- Week definition: Do you want “this week” as Mon–Sun or Sun–Sat?
- Calendars to include: Just your primary calendar, or also any others (e.g., work, shared)? If additional, please name them or share their IDs.
- Working hours: What hours should I treat as your preferred working window (e.g., 9am–5pm)?

Alternatives if the connector continues to fail:
- Share an .ics export for this week, or paste a list of your events (title, date, start/end times, location/meeting link, attendees, and whether they’re optional).
- A screenshot of your weekly view al

---

## GitMCP Server

The GitMCP server provides access to documentation and code repositories through a specialized MCP interface. Unlike connectors, this is a remote MCP server that implements the Model Context Protocol for accessing various code documentation resources.

**Key Features:**
- Documentation search and retrieval
- Code analysis and explanation
- Technical reference access
- Real-time documentation fetching
- Specialized tiktoken documentation access

**Server Details:**
- **Server URL:** `https://gitmcp.io/openai/tiktoken`
- **Authentication:** None required for public documentation
- **Protocol:** HTTP transport
- **Maintained by:** GitMCP (third-party)


ℹ️ **No Authentication Required!**

The GitMCP server for tiktoken documentation is publicly accessible and doesn't require any authentication tokens. This makes it perfect for demonstrating MCP functionality without setup complexity.

In [8]:
# No token required for GitMCP tiktoken documentation server
print("✅ GitMCP server ready - no authentication required!")

✅ GitMCP server ready - no authentication required!


In [9]:
# GitMCP Server Example

def demonstrate_github_mcp_server():
    """
    Demonstrate GitMCP server functionality.
    Shows how to interact with GitHub repositories using the official GitMCP server.
    """
    response = client.responses.create(
        model="gpt-5",
        tools=[{
            "type": "mcp",
            "server_label": "gitmcp",
            "server_url": "https://gitmcp.io/openai/tiktoken",
            "allowed_tools": ["search_tiktoken_documentation", "fetch_tiktoken_documentation"],
            "require_approval": "never"  # Set to "always" for production
        }],
        input="""How does TikToken work?"""
    )
    
    print("🐙 GitHub Analysis Response:")
    print("=" * 50)
    print(response.output_text)
    
    return response

# Demonstrate the GitMCP server
github_response = demonstrate_github_mcp_server()

🐙 GitHub Analysis Response:
tiktoken is OpenAI’s fast tokenizer. It turns text into the integer “tokens” that models consume and back again. Under the hood it uses a byte-level BPE (Byte Pair Encoding) scheme that’s trained for each family of models.

How it works (conceptually)
- Pre-tokenize: A regex (“pat_str”) splits UTF-8 text into chunks that roughly align with words, numbers, whitespace, and punctuation. This speeds up the next step.
- Byte-level BPE merges: Each chunk is treated as a sequence of bytes. Using a learned “merge ranks” table, the algorithm repeatedly merges the most-preferred adjacent byte pairs until no higher-ranked merge applies. The final merged byte sequences map to integer token IDs.
- Reversibility: Decoding is the inverse—token IDs map back to byte sequences and then to the original text. If a sequence wasn’t seen during training, the tokenizer can fall back to smaller units (down to raw bytes), so it always works on arbitrary text.
- Special tokens: Encodi

---

## Complete Summary: OpenAI Responses API with Data and Connectors

This notebook demonstrated the full capabilities of the OpenAI Responses API for building AI applications that can access and reason over external data sources. Here's what we covered:

### 🔍 File Search Capabilities
- **Vector Store Management** - Creating and managing knowledge bases
- **Multi-format Support** - PDFs, documents, code files, and more  
- **Semantic Search** - Finding information by meaning, not just keywords
- **Automatic Citations** - Getting references to source documents
- **Retrieval Customization** - Controlling search results and performance

### 🔌 MCP Servers and Connectors
- **Google Calendar Connector** - Reading and analyzing calendar events with OAuth
- **GitMCP** - Checking out the GitHub Docs!

### 🛡️ Security and Best Practices
- **OAuth Authentication** - Secure token management for external services
- **Approval Controls** - Fine-grained permission management
- **Error Recovery** - Retry logic and graceful failure handling
- **Data Privacy** - Understanding what data is shared with external services